In [2]:
# ---------- installs ----------
!pip -q install timm albumentations opencv-python-headless kaggle

import os, glob, random, gc
import numpy as np
import pandas as pd
import cv2

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

# ---------- paths (너 구조 기준) ----------
TRAIN_CSV = "/content/data/train.csv"
TEST_CSV  = "/content/data/test.csv"
IMG_DIR   = "/content/data/imagesLevelCrossing"

assert os.path.exists(TRAIN_CSV), TRAIN_CSV
assert os.path.exists(TEST_CSV), TEST_CSV
assert os.path.exists(IMG_DIR), IMG_DIR

CANDIDATE_ROOTS = [IMG_DIR, "/content/data", "/content"]

def find_file(filename: str, roots):
    for r in roots:
        p = os.path.join(r, filename)
        if os.path.exists(p):
            return p
    for r in roots:
        hits = glob.glob(os.path.join(r, "**", filename), recursive=True)
        if hits:
            return hits[0]
    raise FileNotFoundError(f"Could not find {filename} under roots={roots}")

# ---------- read csv ----------
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

TARGET_COLS = [
    "probaObstacle1","x1","dx1","y1","dy1",
    "probaObstacle2","x2","dx2","y2","dy2",
    "probaObstacle3","x3","dx3","y3","dy3"
]

presence = (
    (train_df["probaObstacle1"] > 0.5) |
    (train_df["probaObstacle2"] > 0.5) |
    (train_df["probaObstacle3"] > 0.5)
).astype(int).values

# ---------- dataset ----------
class CrossingDataset(Dataset):
    def __init__(self, df: pd.DataFrame, roots, augment=False):
        self.df = df.reset_index(drop=True)
        self.roots = roots

        if augment:
            self.tf = A.Compose([
                A.LongestMaxSize(max_size=768),
                A.PadIfNeeded(min_height=720, min_width=1280, border_mode=cv2.BORDER_CONSTANT),
                A.RandomResizedCrop(size=(384, 704), scale=(0.75, 1.0), ratio=(1.6, 2.0), p=1.0),
                A.HorizontalFlip(p=0.5),
                A.RandomBrightnessContrast(p=0.3),
                A.Normalize(),
                ToTensorV2()
            ])
        else:
            self.tf = A.Compose([
                A.LongestMaxSize(max_size=768),
                A.PadIfNeeded(min_height=720, min_width=1280, border_mode=cv2.BORDER_CONSTANT),
                A.CenterCrop(height=384, width=704),
                A.Normalize(),
                ToTensorV2()
            ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        img_a_path = find_file(row["image-3sec"], self.roots)
        img_b_path = find_file(row["image"], self.roots)
        mask_path  = find_file(row["segmentedRailImage"], self.roots)

        img_a = cv2.cvtColor(cv2.imread(img_a_path), cv2.COLOR_BGR2RGB)
        img_b = cv2.cvtColor(cv2.imread(img_b_path), cv2.COLOR_BGR2RGB)
        mask  = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE).astype(np.float32) / 255.0

        x = np.concatenate([img_a, img_b, mask[..., None]], axis=2).astype(np.float32)  # [H,W,7]
        x_t = self.tf(image=x)["image"]  # [7,H,W]

        if all(c in row.index for c in TARGET_COLS):
            y = torch.tensor(row[TARGET_COLS].astype(np.float32).values, dtype=torch.float32)
            return x_t, y
        else:
            return x_t, int(row["ID"])

# ---------- model ----------
MODEL_NAME = "tf_efficientnetv2_s"

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = timm.create_model(
            MODEL_NAME, pretrained=True, in_chans=7, num_classes=15
        )
    def forward(self, x):
        return self.backbone(x)

# ---------- loss ----------
class MultiObstacleLoss(nn.Module):
    def __init__(self, bbox_weight=5.0, prob_weight=1.0):
        super().__init__()
        self.bce = nn.BCEWithLogitsLoss()
        self.smoothl1 = nn.SmoothL1Loss(reduction="none")
        self.bbox_weight = bbox_weight
        self.prob_weight = prob_weight

    def forward(self, logits, targets):
        p_idx = [0, 5, 10]
        bbox_slices = [(1,5), (6,10), (11,15)]

        prob_logits  = logits[:, p_idx]
        prob_targets = targets[:, p_idx]
        loss_prob = self.bce(prob_logits, prob_targets)

        bbox_loss_total = 0.0
        for i, (s, e) in enumerate(bbox_slices):
            bbox_pred = torch.sigmoid(logits[:, s:e])
            bbox_gt   = targets[:, s:e]
            present = (prob_targets[:, i] > 0.5).float().unsqueeze(1)
            l = self.smoothl1(bbox_pred, bbox_gt)
            bbox_loss_total += (l * present).mean()

        loss = self.prob_weight * loss_prob + self.bbox_weight * bbox_loss_total
        return loss, loss_prob.detach(), bbox_loss_total.detach()

# ---------- train utils ----------
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

seed_everything(42)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
PIN = (device.type == "cuda")

def train_one_epoch(model, loader, optimizer, scaler, criterion):
    model.train()
    total = lp_total = lb_total = 0.0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad(set_to_none=True)
        with torch.cuda.amp.autocast(enabled=PIN):
            logits = model(x)
            loss, lp, lb = criterion(logits, y)

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total += loss.item() * x.size(0)
        lp_total += lp.item() * x.size(0)
        lb_total += lb.item() * x.size(0)

    n = len(loader.dataset)
    return total/n, lp_total/n, lb_total/n

@torch.no_grad()
def valid_one_epoch(model, loader, criterion):
    model.eval()
    total = lp_total = lb_total = 0.0

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss, lp, lb = criterion(logits, y)

        total += loss.item() * x.size(0)
        lp_total += lp.item() * x.size(0)
        lb_total += lb.item() * x.size(0)

    n = len(loader.dataset)
    return total/n, lp_total/n, lb_total/n

# ---------- split ----------
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
tr_idx, va_idx = list(skf.split(train_df, presence))[0]

tr_df = train_df.iloc[tr_idx].reset_index(drop=True)
va_df = train_df.iloc[va_idx].reset_index(drop=True)

train_ds = CrossingDataset(tr_df, CANDIDATE_ROOTS, augment=True)
valid_ds = CrossingDataset(va_df, CANDIDATE_ROOTS, augment=False)

# ✅ 메모리 안전: workers=0, batch 작게
train_loader = DataLoader(train_ds, batch_size=4, shuffle=True, num_workers=0, pin_memory=PIN)
valid_loader = DataLoader(valid_ds, batch_size=4, shuffle=False, num_workers=0, pin_memory=PIN)

model = Net().to(device)
criterion = MultiObstacleLoss().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=6)
scaler = torch.cuda.amp.GradScaler(enabled=PIN)

best = 1e9
EPOCHS = 6
save_path = "/content/best_model.pt"

for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_lp, tr_lb = train_one_epoch(model, train_loader, optimizer, scaler, criterion)
    va_loss, va_lp, va_lb = valid_one_epoch(model, valid_loader, criterion)
    scheduler.step()

    print(f"[{epoch:02d}/{EPOCHS}] "
          f"train {tr_loss:.5f} (prob {tr_lp:.5f}, bbox {tr_lb:.5f}) | "
          f"valid {va_loss:.5f} (prob {va_lp:.5f}, bbox {va_lb:.5f})")

    if va_loss < best:
        best = va_loss
        torch.save({"model": model.state_dict()}, save_path)
        print("saved:", save_path)

    # ✅ epoch 끝날 때 메모리 정리
    gc.collect()
    if PIN:
        torch.cuda.empty_cache()

# ---------- inference ----------
@torch.no_grad()
def predict(model, df):
    ds = CrossingDataset(df, CANDIDATE_ROOTS, augment=False)
    loader = DataLoader(ds, batch_size=4, shuffle=False, num_workers=0, pin_memory=PIN)

    all_ids, all_preds = [], []
    model.eval()

    for x, ids in loader:
        x = x.to(device)
        logits = model(x)
        preds = torch.sigmoid(logits).cpu().numpy()
        all_ids.extend(ids.tolist())
        all_preds.append(preds)

    all_preds = np.concatenate(all_preds, axis=0)
    out = pd.DataFrame(all_preds, columns=TARGET_COLS)
    out.insert(0, "ID", all_ids)
    return out

ckpt = torch.load(save_path, map_location="cpu")
model.load_state_dict(ckpt["model"])

sub = predict(model, test_df)
sub_path = "/content/submission.csv"
sub.to_csv(sub_path, index=False)
print("saved submission:", sub_path)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

/tmp/ipython-input-263993348.py:214: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=PIN)
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torc

[01/6] train 0.24058 (prob 0.22159, bbox 0.00380) | valid 0.21335 (prob 0.19901, bbox 0.00287)
saved: /content/best_model.pt


/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.am

...

[04/6] train 0.11701 (prob 0.11080, bbox 0.00124) | valid 0.07948 (prob 0.06637, bbox 0.00262)


/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.am

[05/6] train 0.10277 (prob 0.09728, bbox 0.00110) | valid 0.05536 (prob 0.03805, bbox 0.00346)
saved: /content/best_model.pt


/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=PIN):
/tmp/ipython-input-263993348.py:163: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.am

[06/6] train 0.09762 (prob 0.09277, bbox 0.00097) | valid 0.07037 (prob 0.05169, bbox 0.00374)
saved submission: /content/submission.csv


In [1]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!mkdir -p /content/data
!kaggle competitions download -c vehicle-stopped-on-a-level-crossing -p /content/data
!unzip -q /content/data/vehicle-stopped-on-a-level-crossing.zip -d /content/data

!ls -al /content/data | head -n 50


100% 1.10G/1.10G [00:11<00:00, 87.6MB/s]
100% 1.10G/1.10G [00:11<00:00, 103MB/s] 
total 1154184
drwxr-xr-x 3 root root       4096 Jan  2 17:54 .
drwxr-xr-x 1 root root       4096 Jan  2 17:53 ..
drwxr-xr-x 2 root root     290816 Jan  2 17:54 imagesLevelCrossing
-rw-r--r-- 1 root root     114710 Aug 21 10:15 sample_submission.csv
-rw-r--r-- 1 root root      42761 Aug 21 10:15 test.csv
-rw-r--r-- 1 root root     238792 Aug 21 10:15 train.csv
-rw-r--r-- 1 root root 1181169582 Aug 21 10:16 vehicle-stopped-on-a-level-crossing.zip


total 1154180
drwxr-xr-x 3 root root       4096 Jan  2 17:44 .
drwxr-xr-x 1 root root       4096 Jan  2 17:43 ..
drwxr-xr-x 2 root root     290816 Jan  2 17:44 imagesLevelCrossing
-rw-r--r-- 1 root root     114710 Aug 21 10:15 sample_submission.csv
-rw-r--r-- 1 root root      42761 Aug 21 10:15 test.csv
-rw-r--r-- 1 root root     238792 Aug 21 10:15 train.csv
-rw-r--r-- 1 root root 1181169582 Aug 21 10:16 vehicle-stopped-on-a-level-crossing.zip
total 1179164
drwxr-xr-x 2 root root 290816 Jan  2 17:44 .
drwxr-xr-x 3 root root   4096 Jan  2 17:44 ..
-rw-r--r-- 1 root root  85668 Aug 21 10:14 0001_1.jpg
-rw-r--r-- 1 root root 106795 Aug 21 10:14 0001_2.jpg
-rw-r--r-- 1 root root   7786 Aug 21 10:14 0001_rail.png
-rw-r--r-- 1 root root  46621 Aug 21 10:14 0003_1.jpg
-rw-r--r-- 1 root root  46621 Aug 21 10:14 0003_2.jpg
-rw-r--r-- 1 root root   5134 Aug 21 10:14 0003_rail.png
-rw-r--r-- 1 root root  43478 Aug 21 10:14 0004_1.jpg
-rw-r--r-- 1 root root  43478 Aug 21 10:14 0004_2.jpg
-rw-r--r